### Iterating Dictionaries (advanced, but not too much)

Recap: iterating a `dict` yields **keys**. Below are practical patterns to iterate keys, values, and key–value pairs, plus ordering, mutation safety, and a few power moves.

## Keys, values, and items

In [1]:
d = {
    'key 1': 1,
    'key 2': 2,
    3.14: 'pi'
}

print('Iterating keys (default):')
for k in d:
    print(k)

print('\nUsing keys() explicitly:')
for k in d.keys():
    print(f"{k} = {d[k]}")

print('\nValues only:')
for v in d.values():
    print(v)

print('\nKey–value pairs via items():')
for k, v in d.items():
    print(f'{k} = {v}')

Iterating keys (default):
key 1
key 2
3.14

Using keys() explicitly:
key 1 = 1
key 2 = 2
3.14 = pi

Values only:
1
2
pi

Key–value pairs via items():
key 1 = 1
key 2 = 2
3.14 = pi


## Views are live (reflect mutations)

In [2]:
items_view = d.items()     # a dynamic view
values_view = d.values()
print('before:', list(items_view))
d['new'] = 42
print('after adding key:', list(items_view))
d['key 2'] = 200
print('values reflect updates too:', list(values_view))

before: [('key 1', 1), ('key 2', 2), (3.14, 'pi')]
after adding key: [('key 1', 1), ('key 2', 2), (3.14, 'pi'), ('new', 42)]
values reflect updates too: [1, 200, 'pi', 42]


## Ordering: insertion order and sorted iteration

In [3]:
ordered = {'a': 1, 'b': 2, 'c': 3}
ordered['x'] = 24  # appended at the end (insertion order)
print('insertion order:')
for k in ordered:
    print(k)

# Sorting by key or by value when you need deterministic custom order
print('\nsorted by key:')
for k in sorted(ordered):
    print(k)

print('\nsorted by value then key:')
for k, v in sorted(ordered.items(), key=lambda kv: (kv[1], kv[0])):
    print(k, v)

insertion order:
a
b
c
x

sorted by key:
a
b
c
x

sorted by value then key:
a 1
b 2
c 3
x 24


## Safe mutation while iterating (copy the keys/items!)

In [4]:
scores = {'alice': 10, 'bob': 0, 'carol': 7, 'dave': 0}

# BAD: mutating the dict you iterate can raise RuntimeError or skip entries
# for k in scores:  # don't do this
#     if scores[k] == 0:
#         del scores[k]

# GOOD: iterate over a static list of keys (or items)
for k in list(scores.keys()):
    if scores[k] == 0:
        del scores[k]
print('after deletion:', scores)

# Another common pattern: build a new dict via comprehension
scores = {'alice': 10, 'bob': 0, 'carol': 7, 'dave': 0}
non_zero = {k: v for k, v in scores.items() if v != 0}
print('filtered copy:', non_zero)

after deletion: {'alice': 10, 'carol': 7}
filtered copy: {'alice': 10, 'carol': 7}


## Iterating nested dictionaries (deep walk and flatten examples)

In [5]:
catalog = {
    'books': {
        'python': {'price': 30, 'stock': 5},
        'data': {'price': 45, 'stock': 2}
    },
    'accessories': {'mug': {'price': 12, 'stock': 20}}
}

print('deep walk:')
for category, items in catalog.items():
    for name, meta in items.items():
        price = meta['price']
        stock = meta['stock']
        print(f"{category}/{name}: ${price} ({stock} left)")

# Flatten to a single-level dict with compound keys
flat = {f"{cat}.{name}": meta for cat, items in catalog.items() for name, meta in items.items()}
print('\nflattened keys:', list(flat.keys()))

deep walk:
books/python: $30 (5 left)
books/data: $45 (2 left)
accessories/mug: $12 (20 left)

flattened keys: ['books.python', 'books.data', 'accessories.mug']


## Views as set-like objects (membership and set ops on keys)

In [6]:
a = {'x': 1, 'y': 2, 'z': 3}
b = {'w': 0, 'y': 20, 'z': 30}

print('membership on keys view:', 'x' in a.keys())
print('common keys:', a.keys() & b.keys())
print('keys only in a:', a.keys() - b.keys())
print('symmetric difference:', a.keys() ^ b.keys())

membership on keys view: True
common keys: {'z', 'y'}
keys only in a: {'x'}
symmetric difference: {'w', 'x'}


## Enumerating with index while iterating items (rare, but sometimes handy)

In [7]:
conf = {'host': 'localhost', 'port': 5432, 'db': 'app'}
for idx, (k, v) in enumerate(conf.items(), start=1):
    print(f"{idx}. {k} = {v}")

1. host = localhost
2. port = 5432
3. db = app


## Transforming dictionaries while iterating (map & filter in one go)

In [8]:
prices = {'banana': 2.5, 'apple': 1.2, 'pear': 2.1, 'cherry': 5.0}

# 10% discount on items priced >= 2.0, keep others as-is
discounted = {
    k: (v * 0.9 if v >= 2.0 else v)
    for k, v in prices.items()
}
print(discounted)

# Keep only items that start with 'p'
p_only = {k: v for k, v in prices.items() if k.startswith('p')}
print(p_only)

{'banana': 2.25, 'apple': 1.2, 'pear': 1.8900000000000001, 'cherry': 4.5}
{'pear': 2.1}


## Re-insertion changes order; updating a value does not (demonstration)

In [9]:
d2 = {'a': 1, 'b': 2, 'c': 3}
d2['b'] = 200   # update keeps position
print('after update:', list(d2.items()))
del d2['b']     # delete removes it
print('after delete:', list(d2.items()))
d2['b'] = 200   # re-insert makes it last
print('after re-insert:', list(d2.items()))

after update: [('a', 1), ('b', 200), ('c', 3)]
after delete: [('a', 1), ('c', 3)]
after re-insert: [('a', 1), ('c', 3), ('b', 200)]


## Bonus: iterating a dict of callables (dispatch tables)

In [10]:
ops = {
    'double': lambda x: x * 2,
    'square': lambda x: x * x,
    'neg': lambda x: -x,
}
x = 5
for name, fn in ops.items():
    print(f"{name}({x}) = {fn(x)}")

double(5) = 10
square(5) = 25
neg(5) = -5
